In [ ]:
!git clone https://github.com/monologg/JointBERT.git

Cloning into 'JointBERT'...
remote: Enumerating objects: 332, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 332 (delta 105), reused 95 (delta 95), pack-reused 213
Receiving objects: 100% (332/332), 487.19 KiB | 1.47 MiB/s, done.
Resolving deltas: 100% (194/194), done.


In [ ]:
def read_data(file_name):
  my_file  = open(file_name,'r')
  data = my_file.read()
  result = data.split("\n")
  my_file.close()
  return result

In [ ]:
train_text = read_data('/content/JointBERT/data/snips/train/seq.in')
train_label = read_data('/content/JointBERT/data/snips/train/label')

In [ ]:
dev_text = read_data('/content/JointBERT/data/snips/dev/seq.in')
dev_label = read_data('/content/JointBERT/data/snips/dev/label')

In [ ]:
test_text = read_data('/content/JointBERT/data/snips/test/seq.in')
test_label = read_data('/content/JointBERT/data/snips/test/label')

In [ ]:
import re
def remove_numbers(lst):
    result = []
    for string in lst:
        result.append(re.sub(r'\d+', '', string))
    return result

In [ ]:
train_text = remove_numbers(train_text)
dev_text = remove_numbers(dev_text)
test_text = remove_numbers(test_text)

In [ ]:
for i in range(len(train_label)):
  if train_label[i]=='':
    print(i)

13084


In [ ]:
train_dict = {
    'text' : train_text[:-1],
    'label' : train_label[:-1]
}
dev_dict = {
    'text' : dev_text[:-1],
    'label' : dev_label[:-1]
}
test_dict = {
    'text' : test_text[:-1],
    'label' : test_label[:-1]
}

In [ ]:
import pandas as pd
train_data = pd.DataFrame(train_dict)
test_data = pd.DataFrame(test_dict)
dev_data = pd.DataFrame(dev_dict)

In [ ]:
train_data.label.value_counts()

PlayMusic               1914
GetWeather              1896
BookRestaurant          1881
RateBook                1876
SearchScreeningEvent    1852
SearchCreativeWork      1847
AddToPlaylist           1818
Name: label, dtype: int64

In [ ]:
test_data.label.value_counts()

AddToPlaylist           124
SearchScreeningEvent    107
SearchCreativeWork      107
GetWeather              104
BookRestaurant           92
PlayMusic                86
RateBook                 80
Name: label, dtype: int64

In [ ]:
dev_data.label.value_counts()

AddToPlaylist           100
BookRestaurant          100
GetWeather              100
PlayMusic               100
RateBook                100
SearchCreativeWork      100
SearchScreeningEvent    100
Name: label, dtype: int64

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:


import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.metrics import accuracy_score

In [ ]:
cats = list(dev_data.label.unique())
cat_dict = {}
for i in range(len(cats)):
  cat_dict[cats[i]]=i

In [ ]:
train_texts = list(train_data.text)[:200]
train_labels = [cat_dict[i] for i in list(train_data.label)][:200]
dev_texts = list(dev_data.text)
dev_labels = [cat_dict[i] for i in list(dev_data.label)]
test_texts = list(test_data.text)
test_labels = [cat_dict[i] for i in list(test_data.label)]

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

# Set device to GPU if available, else CPU
device = tf.device("gpu" if tf.config.list_physical_devices('GPU') else "cpu")

# Define tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=7)

# Define optimizer, loss function, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]

# Define data
# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
dev_encodings = tokenizer(dev_texts, truncation=True, padding=True)

# Convert data to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(len(train_texts)).batch(batch_size=16)
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(batch_size=16)
dev_dataset = tf.data.Dataset.from_tensor_slices((
    dict(dev_encodings),
    dev_labels
)).batch(batch_size=16)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train the model
epochs = 15
history = model.fit(train_dataset, validation_data=dev_dataset, epochs=epochs)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/15
13/13 [==============================] - 71s 812ms/step - loss: 1.8936 - accuracy: 0.2500 - val_loss: 1.7706 - val_accuracy: 0.3857
Epoch 2/15
13/13 [==============================] - 5s 387ms/step - loss: 1.6761 - accuracy: 0.4550 - val_loss: 1.5471 - val_accuracy: 0.6600
Epoch 3/15
13/13 [==============================] - 5s 364ms/step - loss: 1.4400 - accuracy: 0.7650 - val_loss: 1.2414 - val_accuracy: 0.8714
Epoch 4/15
13/13 [==============================] - 5s 414ms/step - loss: 1.0779 - accuracy: 0.9400 - val_loss: 0.9265 - val_accuracy: 0.9000
Epoch 5/15
13/13 [==============================] - 3s 281ms/step - loss: 0.7636 - accuracy: 0.9800 - val_loss: 0.7011 - val_accuracy: 0.9029
Epoch 6/15
13/13 [==============================] - 5s 428ms/step - loss: 0.5468 - accuracy: 0.9800 - val_loss: 0.5064 - val_accuracy: 0.9414
Epoch 7/15
13/13 [==============================] - 3s 277ms/step - loss: 0.3675 - accuracy: 0.9950 - val_loss: 0.3958 - val_accuracy: 0.9486
Epoch